# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import os
import joblib

from azureml.train.automl import AutoMLConfig
from pprint import pprint # Used in printing automl model parameters
from azureml.core import Model # Used to get model information

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-project'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ECVVTUCF3 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


I am using a Gaming dataset found on Kaggle. The dataset consists of 40 columns including "GameId" which is the unique identifier and "BlueWins" which indicates if Blue wins or Red.
I will be performing a binary classification as there are only two possible outcomes.

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

data_loc = "https://raw.githubusercontent.com/AishuDas/nd00333-capstone/master/starter_file/high_diamond_ranked_10min.csv"
ds = TabularDatasetFactory.from_delimited_files(data_loc)
data = ds.to_pandas_dataframe().dropna()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

I refered to the following link for the input parameters. 
https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py
<br>
Upon reading the various parameters I chose the following <br>
1. Primary_metric as accuracy as Accuracy was the objective function <br>
2. I set Max concurrent iterations as 10 to parallize the automl model building process<br>
3. I set experiment timeout to 25 minutes so it doesn't go on building the model for a really long time.<br>
4. I set the task as classification<br>
5. I provided the label column name<br>
6. The training dataset<br>
7. The compute target to be used for computing<br>
8. Cross_validation as 5<br>
I left the rest of the parameters as their default values since they made sense for my run.<br>

In [4]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 10,
    "primary_metric" : 'accuracy'}

computetarget = 'udacity'

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=computetarget,
    task='classification',
    training_data=ds,
    label_column_name='blueWins',   
    n_cross_validations=5,
    **automl_settings
)

In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

{'runId': 'AutoML_3cbe2216-07b8-4f99-b956-f0ffd4ddecbd',
 'target': 'udacity',
 'status': 'Completed',
 'startTimeUtc': '2021-02-03T02:18:54.097331Z',
 'endTimeUtc': '2021-02-03T02:53:15.668267Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'udacity',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-137031","workspace_name":"quick-starts-ws-137031","region":"southcentralus","compute_target":"udacity","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurizati

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: udacity-project,
Id: AutoML_3cbe2216-07b8-4f99-b956-f0ffd4ddecbd_168,
Type: azureml.scriptrun,
Status: Completed)
AUC_weighted 0.8092024090034288
weighted_accuracy 0.7350056047433187
recall_score_micro 0.7349937477579049
f1_score_macro 0.7348756158970338
average_precision_score_macro 0.8065336505947143
f1_score_weighted 0.7350164038385139
precision_score_weighted 0.7351808062949396
AUC_micro 0.8091345088184407
matthews_correlation 0.4698930368353446
precision_score_macro 0.7349113131628237
precision_score_micro 0.7349937477579049
log_loss 0.5340644696045306
accuracy 0.7349937477579049
average_precision_score_micro 0.8065378843081129
recall_score_macro 0.7349817453851457
f1_score_micro 0.7349937477579049
recall_score_weighted 0.7349937477579049
balanced_accuracy 0.7349817453851457
norm_macro_recall 0.4699634907702913
AUC_macro 0.8092024090034288
average_precision_score_weighted 0.8067841873907142
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_3cbe2216-07b8-4

In [12]:
#TODO: Save the best model
best_model = best_run.register_model(model_path='./', model_name='bestModel_automl',tags={'Training context':'Auto ML'},properties={'Accuracy': best_run_metrics['accuracy']})

print(best_model)

Model(workspace=Workspace.create(name='quick-starts-ws-137031', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f', resource_group='aml-quickstarts-137031'), name=bestModel_automl, id=bestModel_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.7349937477579049'})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service